In [ ]:
%matplotlib widget

In [ ]:
from astropy.io import fits
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import IntSlider, interact, VBox
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm

In [ ]:
# Loading the data

name = '/Users/busko/Dropbox/NGC5406.V1200.rscube.fits'
hdulist = fits.open(name)
if len(hdulist)>1:
    hdu=hdulist[1]
else:
    hdu=hdulist[0]

cube = hdu.data.T # remember to always transpose with fits files
hdr = hdu.header
hdr['CRVAL3'] = 0 # adapt to this data
hdr['Cd3_3'] = 1

# Data seems to have lots of garbage at the edges. It has also
# a lot of bad pixels. These should be handled by a masked array
# version of the cube, but for now we resort to clipping and
# forcing the plot axis and grayscales to behave.  

In [ ]:
# Initialize median map
median = np.median(cube, axis=2)

In [5]:
# These classes are being used for experiments with draggable/sizable
# regions. The code was copied as-is from the matplotlib docs:
# https://matplotlib.org/users/event_handling.html
#

class DraggableRectangle:
    def __init__(self, rect, callback):
        self.rect = rect
        self.press = None
        self.callback = callback

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes: return

        contains, attrd = self.rect.contains(event)
        if not contains: return
        print('event contains', self.rect.xy)
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if self.press is None: return
        if event.inaxes != self.rect.axes: return
        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        
        # to fix the Y coordinates of the rectangle
        dy = 0

        #print('x0=%f, xpress=%f, event.xdata=%f, dx=%f, x0+dx=%f' %
        #      (x0, xpress, event.xdata, dx, x0+dx))
        self.rect.set_x(x0+dx)
        self.rect.set_y(y0+dy)

        self.rect.figure.canvas.draw()

    def on_release(self, event):
        'on release we reset the press data'
        self.press = None
        self.rect.figure.canvas.draw()
        
        self.callback(self.rect)

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)

# draggable rectangle with the animation blit techniques; see
# http://www.scipy.org/Cookbook/Matplotlib/Animations
class DraggableRectangle_Blit:
    lock = None  # only one can be animated at a time
    def __init__(self, rect, callback):
        self.rect = rect
        self.press = None
        self.background = None
        self.callback = callback

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes: return
        if DraggableRectangle_Blit.lock is not None: return
        contains, attrd = self.rect.contains(event)
        if not contains: return
        print('event contains', self.rect.xy)
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata
        DraggableRectangle_Blit.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        self.rect.set_animated(True)
        canvas.draw()
        self.background = canvas.copy_from_bbox(self.rect.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.rect)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if DraggableRectangle_Blit.lock is not self:
            return
        if event.inaxes != self.rect.axes: return
        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        
        # to fix the Y coordinates of the rectangle
        dy = 0
        
        self.rect.set_x(x0+dx)
        self.rect.set_y(y0+dy)

        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.rect)

        # blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_release(self, event):
        'on release we reset the press data'
        if DraggableRectangle_Blit.lock is not self:
            return

        self.press = None
        DraggableRectangle_Blit.lock = None

        # turn off the rect animation property and reset the background
        self.rect.set_animated(False)
        self.background = None

        # redraw the full figure
        self.rect.figure.canvas.draw()
        
        self.callback(self.rect)

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)

In [6]:
# Create figure and axes
fig_cube = plt.figure(figsize=(8,5))
ax_cube = fig_cube.add_subplot(221)
ax_median = fig_cube.add_subplot(222)
ax_plot = fig_cube.add_subplot(223)

ax_cube.set_xticks([])
ax_cube.set_yticks([])
ax_median.set_xticks([])
ax_median.set_yticks([])
ax_cube.set_title('3D Data Cube')
ax_median.set_title('Median Map')

# Display the raw cube and median maps
median_im = ax_median.imshow(median, vmin=0.,vmax=0.02)
divider = make_axes_locatable(ax_median)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig_cube.colorbar(median_im, cax=cax)

cube_im = ax_cube.imshow(cube[:,:,0],vmin=0.,vmax=0.02,cmap=cm.get_cmap('Greys'))
divider = make_axes_locatable(ax_cube)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig_cube.colorbar(cube_im, cax=cax)

plt.tight_layout()

# Create the slider and its callback
slider = IntSlider(value=int(cube.shape[2]/2.), min=0, max=cube.shape[2]-1)

def plot_test(slice_num):
    cube_im.set_data(cube[:,:,slice_num])
    fig_cube.canvas.draw_idle()

_ = interact(plot_test, slice_num=slider)

# Plot spectrum in the center of the cube, and add
# the 1D plotting interactivity to the cube image.
ax_plot.set_ylim([0.0,0.1])
ax_plot.plot(cube[int(cube.shape[0]/2),int(cube.shape[1]/2),:])

def onclick(event):
    if event.inaxes == ax_cube:
        ax_plot.clear()
        ax_plot.set_ylim([0.0,0.1])
        ax_plot.plot(cube[int(event.xdata),int(event.ydata),:])
        
cid = fig_cube.canvas.mpl_connect('button_press_event', onclick)
#VBox([fig_cube.canvas, slider]) # this is needed to display the widget under voila

# experiments with a draggable region
def recalculate_median(new_rect):
    print("HERE!")
    median = np.median(cube, axis=2)
    
rect = ax_plot.bar((750.), 0.1, width=500., color='#999999', edgecolor='k')

# the blit version of the animation has some issues with colors and 
# alphas. The non-blit version is very slow.
#dr = DraggableRectangle_Blit(rect, recalculate_median)
dr = DraggableRectangle(rect[0], recalculate_median)

dr.connect()

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=850, description='slice_num', max=1700), Output()), _dom_classes=('widge…

event contains (500.0, 0)
HERE!
